# Data Preprocessing

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

# reading the dataset and splitting into target and features
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

# label encoding the categorical values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

# one hot encoding
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))

# splitting the dataset into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# standardizing the dataset
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building ANN

In [3]:
# building the model
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [4]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [5]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100


2023-01-10 12:44:42.149866: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


250/250 [==============================] - 0s 470us/step - loss: 0.5791 - accuracy: 0.7496
Epoch 2/100
250/250 [==============================] - 0s 447us/step - loss: 0.4833 - accuracy: 0.7964
Epoch 3/100
250/250 [==============================] - 0s 437us/step - loss: 0.4502 - accuracy: 0.8065
Epoch 4/100
250/250 [==============================] - 0s 435us/step - loss: 0.4360 - accuracy: 0.8142
Epoch 5/100
250/250 [==============================] - 0s 432us/step - loss: 0.4257 - accuracy: 0.8220
Epoch 6/100
250/250 [==============================] - 0s 429us/step - loss: 0.4180 - accuracy: 0.8256
Epoch 7/100
250/250 [==============================] - 0s 442us/step - loss: 0.4122 - accuracy: 0.8269
Epoch 8/100
250/250 [==============================] - 0s 433us/step - loss: 0.4075 - accuracy: 0.8273
Epoch 9/100
250/250 [==============================] - 0s 430us/step - loss: 0.4027 - accuracy: 0.8275
Epoch 10/100
250/250 [==============================] - 0s 434us/step - loss: 0.3985 

Epoch 80/100
250/250 [==============================] - 0s 434us/step - loss: 0.3288 - accuracy: 0.8627
Epoch 81/100
250/250 [==============================] - 0s 428us/step - loss: 0.3287 - accuracy: 0.8648
Epoch 82/100
250/250 [==============================] - 0s 429us/step - loss: 0.3288 - accuracy: 0.8649
Epoch 83/100
250/250 [==============================] - 0s 427us/step - loss: 0.3291 - accuracy: 0.8650
Epoch 84/100
250/250 [==============================] - 0s 429us/step - loss: 0.3288 - accuracy: 0.8637
Epoch 85/100
250/250 [==============================] - 0s 427us/step - loss: 0.3283 - accuracy: 0.8660
Epoch 86/100
250/250 [==============================] - 0s 424us/step - loss: 0.3285 - accuracy: 0.8645
Epoch 87/100
250/250 [==============================] - 0s 428us/step - loss: 0.3281 - accuracy: 0.8649
Epoch 88/100
250/250 [==============================] - 0s 427us/step - loss: 0.3284 - accuracy: 0.8649
Epoch 89/100
250/250 [==============================] - 0s 427us

# Testing the Model

**Example**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

In [6]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 63ms/step
[[False]]


**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

In [10]:
# predicting outputs from test features
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

63/63 [==============================] - 0s 383us/step


In [11]:
# calculating accuracy
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1524   71]
 [ 199  206]]


0.865